In [43]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import lazynlp
import timeout_decorator
import htmldate
from tqdm import tqdm

In [15]:
# Temporary block to combine individual files since my parsing failed part of the way through ;-;
'''articlesEN = np.load("./data/articlesEN.npy", allow_pickle=True)
articlesRU = np.load("./data/articlesRU.npy", allow_pickle=True)
articlesZH = np.load("./data/articlesZH.npy", allow_pickle=True)

abvs = ["en", "ru", "zh"]
article_dicts = [articlesEN, articlesRU, articlesZH]
articles = {}

for i in range(len(abvs)):
  abv = abvs[i]
  article_dict = article_dicts[i]
  filtered_articles = []
  for article in article_dict:
    if len(article["text"]) > 100:
      filtered_dict = {"url": article["url"], "title": article["title"], 
                     "authors": article["authors"], "date": article["date"],
                     "text": article["text"]}
      filtered_articles.append(filtered_dict)
  articles[abv] = filtered_articles
  print("Number of " + abv + " articles:" + str(len(filtered_articles)))
  np.save("./data/articles.npy", articles)'''

'articlesEN = np.load("./data/articlesEN.npy", allow_pickle=True)\narticlesRU = np.load("./data/articlesRU.npy", allow_pickle=True)\narticlesZH = np.load("./data/articlesZH.npy", allow_pickle=True)\n\nabvs = ["en", "ru", "zh"]\narticle_dicts = [articlesEN, articlesRU, articlesZH]\narticles = {}\n\nfor i in range(len(abvs)):\n  abv = abvs[i]\n  article_dict = article_dicts[i]\n  filtered_articles = []\n  for article in article_dict:\n    if len(article["text"]) > 100:\n      filtered_dict = {"url": article["url"], "title": article["title"], \n                     "authors": article["authors"], "date": article["date"],\n                     "text": article["text"]}\n      filtered_articles.append(filtered_dict)\n  articles[abv] = filtered_articles\n  print("Number of " + abv + " articles:" + str(len(filtered_articles)))\n  np.save("./data/articles.npy", articles)'

In [38]:
# Filter out articles with < 100 characters) and print number of articles for each language
articles = np.load("./data/articles.npy", allow_pickle=True).item()
for abv, article_list in articles.items():
  filtered_articles = []
  for article in article_list:
    if len(article["text"]) > 100:
      filtered_articles.append(article)
  articles[abv] = filtered_articles
  print("Number of " + abv + " articles:" + str(len(filtered_articles)))

#np.save("./data/filtered_articles.npy", articles)

Number of en articles:1906
Number of ru articles:1431
Number of zh articles:9627


In [39]:
def lazy_clean_txt(txt):
  txt = lazynlp.clean_html(txt)
  txt = lazynlp.transliterate(txt)
  txt = lazynlp.collapse_white_spaces(txt)
  txt = lazynlp.connect_lines(txt)
  txt = lazynlp.replace_unprintable(txt)
  return txt

In [20]:
articles = np.load("./data/articles.npy", allow_pickle=True).item()

# Extra text clearning step using lazynlp
for lang, article_list in articles.items():
  for article in tqdm(article_list):
    article["text"] = lazy_clean_txt(article["text"])

100%|██████████| 9627/9627 [00:11<00:00, 848.08it/s] 


In [44]:
utc=pytz.UTC

@timeout_decorator.timeout(5, timeout_exception=StopIteration)
def standardize_date(article):
  # Try parsing date using htmldate if it's currently none
  if article["date"] is None:
    try:
      article["date"] = datetime.strptime(htmldate.find_date(article["url"]), "%Y-%m-%d")
    except Exception:
      print("Error while trying to manually get datetime")
  # If date is not none (after trying htmldate), then convert it to UTC
  if article["date"] is not None:
    article["date"] = article["date"].replace(tzinfo=utc)

In [45]:
# Remove all tab and newline characters from text:
# Convert all datetimes to UTC:
for lang, article_list in articles.items():
  for article in tqdm(article_list):
    article["text"] = article["text"].replace("\t", "").replace("\n", " ")
    standardize_date(article)

# Convert everything in the articles.npy file to a pandas dataframe
article_dfs = {}
for lang in articles.keys():
  lang_df = pd.DataFrame(articles[lang])
  article_dfs[lang] = lang_df
  lang_df.to_csv("./data/" + lang + "_articles.tsv", index=False, sep="\t")

100%|██████████| 1906/1906 [00:00<00:00, 9563.52it/s]


Error while trying to manually get datetime


 93%|█████████▎| 1333/1431 [00:00<00:00, 1547.59it/s]

Error while trying to manually get datetime
Error while trying to manually get datetime


100%|██████████| 1431/1431 [00:10<00:00, 131.42it/s] 


...

 31%|███       | 2980/9627 [07:56<54:37,  2.03it/s]

Error while trying to manually get datetime


 72%|███████▏  | 6892/9627 [20:25<34:42:20, 45.68s/it]

Error while trying to manually get datetime


100%|██████████| 9627/9627 [21:10<00:00,  7.58it/s]   


In [ ]:
SAMPLE_SIZE = 50
SAMPLE_YEAR = 2021

# Randomly sample 50 articles from each month in 2022 (if there are that many)

articles_month_dict = {}

for lang, article_list in articles.items():
  articles_by_month = []
  for month in range(12):
    articles_by_month.append([])

  for article in article_list:
    article_date = article["date"]
    if (article_date is not None and article_date.year == SAMPLE_YEAR):
      articles_by_month[article_date.month - 1].append(article)

  articles_month_dict[lang] = articles_by_month

# Vast vast majority of articles are from the most recent month... which makes sense
# but also means there's probably not enough data to sample accross multiple months into the past
for lang in ["zh", "ru", "en"]:
  articles_sampled_by_month = np.empty((12, SAMPLE_SIZE), dtype=object)
  print(lang + ": ")
  for month in range(12):
    print(len(articles_month_dict[lang][month]))